In [56]:
import os, json, requests
from requests.auth import HTTPBasicAuth
from pprint import pprint
import pdb
os.environ['PL_API_KEY']  = '7d8af35b6e944f33bb5e33ada32ab4a0'
PL_API_KEY = os.getenv('PL_API_KEY')
PL_AUTH = HTTPBasicAuth(PL_API_KEY, '')

# Setup Planet Data API base URL
BASE_URL = "https://api.planet.com/data/v1"
STATS_URL = f'{BASE_URL}/stats'

#Setup the session
sess = requests.Session()
# Authenticate
sess.auth = (PL_API_KEY, '')

# printout helper
def p(response):
    pprint(response.json())

In [26]:
# Try a simple request using the session
res = sess.get(BASE_URL)
p(res)

{'_links': {'_self': 'https://api.planet.com/data/v1/',
            'asset-types': 'https://api.planet.com/data/v1/asset-types/',
            'item-types': 'https://api.planet.com/data/v1/item-types/',
            'spec': 'https://api.planet.com/data/v1/spec'}}


In [27]:
temp = sess.get(res.json()['_links']['item-types'])

In [37]:
item_type_ids = [itype['id'] for itype in temp.json()['item_types']]
print(len(item_type_ids))
pprint(item_type_ids)

14
['MYD09GQ',
 'PSScene4Band',
 'SkySatScene',
 'PSScene3Band',
 'Sentinel1',
 'REScene',
 'REOrthoTile',
 'Sentinel2L1C',
 'MOD09GA',
 'MYD09GA',
 'SkySatCollect',
 'PSOrthoTile',
 'Landsat8L1G',
 'MOD09GQ']


In [47]:
item_type = 'PSScene4Band'


In [21]:
# Filters

<Response [405]>

In [43]:
## Set Area of Interest
aoi_data = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -118.2967472076416,
              34.00578318672437
            ],
            [
              -118.22739601135252,
              34.00578318672437
            ],
            [
              -118.22739601135252,
              34.07846940942791
            ],
            [
              -118.2967472076416,
              34.07846940942791
            ],
            [
              -118.2967472076416,
              34.00578318672437
            ]
          ]
        ]
      }
    }
  ]
}
aoi_geom = aoi_data['features'][0]['geometry']

In [44]:
geom_filter = {
    'type': 'GeometryFilter',
    'field_name': 'geometry',
    'config': aoi_geom
}

date_range_filter = {
    'type': 'DateRangeFilter',
    
    #The field to filter on: 'acquired` meaning, 
    #"date on which the scene image was captured"
    'field_name': 'acquired', 
    'config': {
        "gte": "2016-01-01T00:00:00.000Z",
        "lte": "2019-01-01T00:00:00.000Z"
    }
}

cloud_cover_filter = {
    'type':'RangeFilter',
    'field_name': 'cloud_cover',
    'config': {
        "lte":0.1
    }
}

combined_filter = {
    'type': 'AndFilter',
    'config': [geom_filter, date_range_filter, cloud_cover_filter]
}

In [48]:
# Construct the request
search_request = {
    'item_types': [item_type],
    'interval': 'month',
    'filter': combined_filter
}

# Send the post request to the quick search endpoint
SEARCH_URL = f'{BASE_URL}/quick-search'
search_resp = sess.post(SEARCH_URL, json=search_request)

In [50]:
search_res = search_resp.json()['features']
print(len(search_res)) # number of images satifying the filter requirements

250


In [53]:
search_resp.json()['_links']

{'_first': 'https://api.planet.com/data/v1/searches/22650a3f529549c2b9814f78548f9b77/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D',
 '_next': 'https://api.planet.com/data/v1/searches/22650a3f529549c2b9814f78548f9b77/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6ICIyMDE4LTA5LTA5VDA2OjA2OjIwLjY4MTAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMTgwOTA4XzE3NTkzNV8xMDUxIiwgInNvcnRfZGVzYyI6IHRydWV9',
 '_self': 'https://api.planet.com/data/v1/searches/22650a3f529549c2b9814f78548f9b77/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D'}

In [66]:
ids_page1 = [f['id'] for f in search_resp.json()['features']]
print(len(ids_page1))
pprint(ids_page1[::10])

250
['20181230_181227_0f4e',
 '20181228_180349_0e26',
 '20181219_180938_1018',
 '20181213_181058_103e',
 '20181207_174643_104d',
 '20181202_180839_0f25',
 '20181123_174924_0f4a',
 '20181120_180854_1018',
 '20181115_174949_0f4a',
 '20181106_175144_0f36',
 '20181103_180850_101b',
 '20181101_180756_1002',
 '20181030_180929_0f17',
 '20181026_181011_1018',
 '20180225_174711_1105',
 '20181019_180748_1003',
 '20181015_180624_1029',
 '20181009_180828_0f17',
 '20181001_180731_1033',
 '20180924_175747_104d',
 '20180922_175659_1050',
 '20180920_175734_101c',
 '20180915_175841_0f44',
 '20180912_180708_1015',
 '20180910_180659_0f42']


In [81]:
# first_url = search_resp.json()['_links']['_first']
resp = sess.get(first_url).json()
# p(resp)
ids_first = [f['id'] for f in resp['features']]
print(len(ids_first))
pprint(ids_first[::10])

250
['20181230_181227_0f4e',
 '20181228_180349_0e26',
 '20181219_180938_1018',
 '20181213_181058_103e',
 '20181207_174643_104d',
 '20181202_180839_0f25',
 '20181123_174924_0f4a',
 '20181120_180854_1018',
 '20181115_174949_0f4a',
 '20181106_175144_0f36',
 '20181103_180850_101b',
 '20181101_180756_1002',
 '20181030_180929_0f17',
 '20181026_181011_1018',
 '20180225_174711_1105',
 '20181019_180748_1003',
 '20181015_180624_1029',
 '20181009_180828_0f17',
 '20181001_180731_1033',
 '20180924_175747_104d',
 '20180922_175659_1050',
 '20180920_175734_101c',
 '20180915_175841_0f44',
 '20180912_180708_1015',
 '20180910_180659_0f42']


In [82]:
print(resp['_links']['_next'] == search_resp['_links']['_next'])

True


In [79]:
# search_resp = search_resp.json()

In [99]:
ids = ids_page1
count = 0
try:# whenever you hit this next url, the next page is returned (like a generator)
    next_url = resp['_links']['_next'] 
    print(count, next_url)
    
    # assign to this page's result - this enables to go through all pages
    resp = sess.get(next_url).json()
    ids_next = [f['id'] for f in resp['features']]
    ids.append(ids_next)
    count += 1
#     pprint(ids_next[::10])
except Exception as e:
    pdb.set_trace()
    print(e, count)
    print('end of page iteration')

0 https://api.planet.com/data/v1/searches/22650a3f529549c2b9814f78548f9b77/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6ICIyMDE3LTEwLTI4VDA2OjQxOjM5LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMTcwOTA2XzE4NTM0NF8xMDQ5IiwgInNvcnRfZGVzYyI6IHRydWV9


In [93]:
print('total retrieved scene numbers: ', len(ids))

total retrieved scene numbers:  250


In [75]:
# whenever you hit this next url, the next page is returned (like a generator)
next_url = search_resp.json()['_links']['_next'] 
next_resp = sess.get(next_url).json()
ids_next = [f['id'] for f in next_resp['features']]
print(len(ids_next))
pprint(ids_next[::10])


250
['20180907_180655_100e',
 '20180831_180556_1034',
 '20180828_180728_0f51',
 '20180824_180346_0e2f',
 '20180820_180652_0f12',
 '20180817_180211_1050',
 '20180815_180617_0f43',
 '20180811_180421_1048',
 '20180808_180355_104e',
 '20180805_180639_0f4e',
 '20180801_180625_101e',
 '20180728_180620_1038',
 '20180723_180516_103e',
 '20180721_180224_0e26',
 '20180715_180710_0f4b',
 '20180712_180801_0f2d',
 '20180707_180914_1043',
 '20180706_180453_103e',
 '20180703_180433_1004',
 '20180628_180443_1012',
 '20180626_180528_0f22',
 '20161004_004839_0c62',
 '20180619_180345_103e',
 '20180614_180313_101f',
 '20180611_180437_101e']


### Saved Searches

In [115]:
SAVED_SEARCH_URL = f"{BASE_URL}/searches"
# add the name field which specifies the name of the saved search
search_request2 = {
    "name": "downtown_la",
    "interval": "day",
    "item_types": [item_type],
    'filter': combined_filter
}

# send a post request to the sdaved searches endpoint to create a saved search
res = sess.post(SAVED_SEARCH_URL, 
                json=search_request2)
p(res)

{'__daily_email_enabled': False,
 '_links': {'_self': 'https://api.planet.com/data/v1/searches/adb56abeaffb42858e4c13898388d6a3',
            'results': 'https://api.planet.com/data/v1/searches/adb56abeaffb42858e4c13898388d6a3/results'},
 'created': '2019-05-08T01:11:12.005177Z',
 'filter': {'config': [{'config': {'coordinates': [[[-118.2967472076416,
                                                     34.00578318672437],
                                                    [-118.22739601135252,
                                                     34.00578318672437],
                                                    [-118.22739601135252,
                                                     34.07846940942791],
                                                    [-118.2967472076416,
                                                     34.07846940942791],
                                                    [-118.2967472076416,
                                                     34.0057


todo:
- [ ] download LA DOWNTOWN IMAGES FROM THIS DATERANGE AND SAVE TO AN . IMAGE
- [ ] CHANGE geometry tO paris - notredame within the last year range and download/save the images
- [ ] Make a timelapse with those images 
    - [ ] gdal: 16bits tiff => jpeg